## NPR SUNDAY PUZZLE January 31, 2021
https://www.npr.org/2021/01/31/962412357/sunday-puzzle-game-of-words

> This week's challenge: This week's challenge comes from listener Derrick Niederman, of Charleston, S.C. Starting in Montana, you can drive into South Dakota and then into Iowa. Those three states have the postal abbreviations MT, SD, and IA — whose letters can be rearranged to spell AMIDST. The challenge is to do this with four connected states to make an eight-letter word. That is, start in a certain state, drive to another, then another, and then another. Take the postal abbreviations of the four states you visit, mix the letters up, and use them to spell a common eight-letter word. Derrick and I know of only one answer. Can you do this?

Sources:
* List of common 8 letter words: http://www.poslarchive.com/math/scrabble/lists/common-8.html
* Table of state borders: Thomas J. Holmes (April 30, 1998) http://users.econ.umn.edu/~holmes/data/BorderData.html

Author: William Seifstad



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import networkx as nx

In [2]:
# import data from csv to a pandas table
df = pd.read_csv('state_borders.csv')

# import 8 letter words
with open("eight_letter_words.txt", "r") as f:
    words = f.read().split(" ")
    words = [re.split(r'[\n\t]', i) for i in words]
    words = [item for sublist in words for item in sublist]

In [3]:
# cleaning the state borders table data
df['state1'] = df['ST1ST2'].apply(lambda x: x.split('-')[0])
df['state2'] = df['ST1ST2'].apply(lambda x: x.split('-')[1])
df = df[['state1','state2']]

# reversing and concatenating the columsn, since the original table only contains unique border pairs
# (I need a table of all borders, not just unique borders) 
df2 = df.copy()
df2['state1'] = df['state2'].copy()
df2['state2'] = df['state1'].copy()
df = pd.concat([df,df2],axis=0)

In [4]:
# make border pairs into dictionary to work nicely with networkx graph
borders = pd.Series.to_dict(df.groupby('state1')['state2'].apply(list))

# generate network x graph, using the state border pairs as edges and the states as nodes
g = nx.Graph(borders)

# iterate through all key - key pairs and get all possible paths of length 4
possible_paths = []
for j in borders.keys():
    for k in borders.keys():
        if j != k:
            possible_paths.extend([list(i) for i in list(nx.all_simple_paths(g,source=j,target=k,cutoff=3)) if len(i) == 4])            

# print(len(possible_paths)) ## contains duplicates
# 3310

In [5]:
# organizing paths as dataframe for deduplication across paths and pretty output
p = pd.DataFrame(possible_paths,columns = [1,2,3,4])
p['path_string'] = p[1] + '-' + p[2] + '-' + p[3] + '-' + p[4]
p['unique_path'] = (p[1] + p[2] + p[3] + p[4]).apply(lambda x: ''.join(sorted(x)))
p.drop_duplicates('unique_path',inplace=True)

In [6]:
solutions = []
for i in p.unique_path:
    for j in words:
        if i.lower() == ''.join(sorted(j)):
            path = p[p['unique_path'] == i]['path_string'].values[0]
            word = j
            solutions.append([path,word])

In [7]:
print(len(solutions),'solutions:')
for i in solutions:
    print(i[0],'-->',i[1])

36 solutions:
AL-TN-MO-AR --> matronal
AL-TN-MO-NE --> nonmetal
AL-GA-TN-VA --> galavant
AR-MO-NE-CO --> coenamor
AR-TN-GA-FL --> flagrant
AR-MO-TN-GA --> martagon
AR-MS-TN-GA --> tangrams
AR-MS-TN-GA --> trangams
AR-TN-MO-IA --> animator
AR-TN-MO-NE --> ornament
AR-MO-IA-SD --> dioramas
AR-MO-NE-SD --> madrones
AR-MO-NE-SD --> ransomed
CO-KS-MO-IA --> oomiacks
CO-NE-SD-IA --> codeinas
CO-NE-SD-IA --> diocesan
CO-NE-IA-MN --> monecian
CO-NE-SD-MN --> condemns
DE-PA-OH-IN --> diaphone
GA-TN-KY-IL --> takingly
GA-TN-MO-NE --> magneton
IA-SD-NE-MO --> amidones
IA-SD-NE-MO --> daimones
IA-SD-NE-MO --> domaines
IA-NE-SD-MT --> mediants
IA-MO-OK-NM --> makimono
IA-NE-MO-TN --> antinome
IA-NE-MO-TN --> nominate
ID-UT-CO-NE --> eduction
IL-MO-NE-KS --> moleskin
IL-MO-AR-MS --> moralism
MN-SD-IA-MO --> monadism
MN-SD-IA-MO --> nomadism
MO-IA-SD-ND --> diamonds
SD-IA-MO-TN --> saintdom
SD-NE-CO-UT --> contused
